In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
# WIDGETS
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

In [19]:
# plot
import matplotlib.pyplot as plt
import bg_mpl_stylesheet
from bg_mpl_stylesheet.bg_mpl_stylesheet import bg_mpl_style
plt.style.use(bg_mpl_style)
plt.ion()
%matplotlib qt

In [4]:
# change to inline
%matplotlib inline

In [5]:
def get_x_fitting_range(x, y, xrange: list):
    # x bound
    if xrange[0] == "min":
        xmin = float(x.min())
    else:
        xmin = float(xrange[0])
    if xrange[1] == "max":
        xmax = float(x.max())
    else:
        xmax = float(xrange[1])
    assert all([isinstance(xmin, float), isinstance(xmax, float)])
    
    bool_range = np.logical_and((x <= xmax), (x >= xmin))
    _x = x[bool_range]
    _y = list()
    for i, yi in enumerate(y):
        yi = yi[bool_range]
        _y.append(yi)
    return _x, np.array(_y)

In [6]:
def xy_dim_labels(xy_lable: str):
    dim_y = xy_lable[0]
    dim_x = xy_lable[1]
    if dim_y == 'i':
        dim_y = 'I (a.u.)'
    elif dim_y == 's':
        dim_y = 'S ($\mathrm{\AA^{-1}})$'
    elif dim_y == 'f':
        dim_y = 'F ($\mathrm{\AA^{-1}})$'
    elif dim_y == 'g':
        dim_y = 'G ($\mathrm{\AA^{-2}})$'
    if dim_x == 'q':
        dim_x = 'Q ($\mathrm{\AA^{-1}})$'
    elif dim_x == 'r':
        dim_x = 'r ($\mathrm{\AA})$'
    return dim_x, dim_y

In [7]:
# archive  
def _plot(files, xrange, yshift):
    fig, ax = plt.subplots(len(files), num='fits')
    if not isinstance(ax, list):
        ax = [ax]
    for i, fname in enumerate(files):
        # collect data 
        dat = np.loadtxt(fname).T
        x = dat[0]
        y = dat[1:4]
        x, y = get_x_fitting_range(x, y, xrange)    
        r = x
        g, diff, fit = y[0:3]
        # plot
        ax[i].set_title(f"Fit:  {fname}")
        dim_x, dim_y = xy_dim_labels(data_type.value)
        ax[i].set_xlabel(dim_x)
        ax[i].set_ylabel(dim_y)
        ax[i].set_xlim(r.min(),r.max())

        ## data
        ax[i].plot(r, np.zeros(len(r)), alpha=0.5, c='C4')
        ax[i].plot(r, g, alpha=0.8, marker='o', fillstyle='none', markersize=7,  c="C0", label='data')

        ## fit
        ax[i].plot(r, fit, alpha=0.8, c="C1", label='fit')

        ## diff
        ax[i].plot(r, np.zeros(len(r)) + (yshift), alpha=0.7, c='C4')
        ax[i].plot(r, diff + (yshift), alpha=0.8, c="C2", label='diff')
        ax[i].legend()


In [8]:
# minv = widgets.FloatSlider(min=x.min(), max=x.max(), value=x.min(), continuous_update= True)
# maxv = widgets.FloatSlider(min=x.min(), max=x.max(), value=x.max(), continuous_update= True)
# diff_shift = widgets.FloatSlider(min=-5, max=5, value=0, step=0.1, continuous_update= True)

# ui = widgets.VBox([widgets.HBox([widgets.Label("min :"), minv]), 
#                          widgets.HBox([widgets.Label("max :"), maxv]),
#                          widgets.HBox([widgets.Label("diff shift"),diff_shift])
#                         ])
# def f(minv, maxv, diff_shift):
#     print((minv, maxv, diff_shift))

# config = widgets.interactive_output(f, {'minv': minv, 'maxv': maxv, 'diff_shift': diff_shift})

# display(ui)

In [9]:
# # setup 
# xmin, xmax, yshift = list(map(eval,config.outputs[0]['text'].strip().strip('()').split(',')))
# xrange = [xmin, xmax]

In [21]:
# plot
def plot(fname, data_type, xmin, xmax, yshift):
    plt.clf()
    fig, ax = plt.subplots(num='fits')

    # collect data 
    dat = np.loadtxt(fname).T
    x = dat[0]
    y = dat[1:4]
    x, y = get_x_fitting_range(x, y, [xmin, xmax])    
    
    # assign
    r = x
    g, diff, fit = y[0:3]
    
    # plot
    ax.set_title(f"Fit:  {fname}")
    dim_x, dim_y = xy_dim_labels(data_type)
    ax.set_xlabel(dim_x)
    ax.set_ylabel(dim_y)
    ax.set_xlim(r.min(),r.max())

    ## data
    ax.plot(r, np.zeros(len(r)), alpha=0.5, c='C4')
    ax.plot(r, g, alpha=0.8, marker='o', fillstyle='none', markersize=7,  c="C0", label='data')

    ## fit
    ax.plot(r, fit, alpha=0.8, c="C1", label='fit')

    ## diff
    ax.plot(r, np.zeros(len(r)) + (yshift), alpha=0.7, c='C4')
    ax.plot(r, diff + (yshift), alpha=0.8, c="C2", label='diff')
    ax.legend()


In [11]:
# setup

dirpath = widgets.Text('./')
w_dirpath = widgets.HBox([widgets.Label("dirpath:"), dirpath])


# data_type = widgets.Dropdown(options=['gr', 'fq', 'sq', 'iq'])
# w_data_type = widgets.HBox([widgets.Label("data type:"), data_type])

path = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True)
w_path = widgets.HBox([widgets.Label("files :"), path])

display(w_dirpath, w_path)

In [12]:
os.chdir(dirpath.value)
files = list(path.value.keys())
dat = np.loadtxt(files[0]).T
x = dat[0]

In [22]:
interactive_plot = interactive(plot, 
                               fname = files, 
                               data_type = ['gr', 'fq', 'sq', 'iq'], 
                               xmin=(x.min()-x.min()/10, x.max()), 
                               xmax = (x.min(),x.max()+x.max()/10), 
                               yshift=(-5,5,0.1))
output = interactive_plot.children[-1]

In [23]:
interactive_plot

interactive(children=(Dropdown(description='fname', options=('example.dat',), value='example.dat'), Dropdown(d…